In [1]:
import pandas as pd
import os
import ast
import json
import csv
import re
from tqdm import tqdm

In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
def clean_json_text(s: str) -> str:
    """
    1. Remove outer quotes if the whole string is wrapped in one pair of quotes.
    2. Escape unescaped newlines inside strings; replace bare newlines with spaces.
    3. Escape other control characters as Unicode escapes.
    4. Collapse illegal double-quoted values (""foo"" → "foo").
    """
    if not isinstance(s, str) or not s.strip():
        return s

    # 1) Unwrap outer quotes
    if len(s) >= 2 and s[0] == '"' and s[-1] == '"':
        s = s[1:-1]

    out = []
    in_string = False
    esc = False
    for ch in s:
        if esc:
            out.append(ch)
            esc = False
        elif ch == '\\':
            out.append(ch)
            esc = True
        elif ch == '"' and not esc:
            in_string = not in_string
            out.append(ch)
        elif ch in ('\n', '\r'):
            out.append('\\n' if in_string else ' ')
        elif ord(ch) < 0x20:
            out.append(f'\\u{ord(ch):04x}')
        else:
            out.append(ch)
    fixed = ''.join(out)
    # 4) Collapse ""foo"" → "foo"
    fixed = re.sub(r'""([^\"]*?)""', r'"\1"', fixed)
    return fixed


def process_json_string(json_str):
    """Fix embedded JSON strings (e.g. summary field)"""
    if not json_str or not isinstance(json_str, str):
        return {}
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        # fallback: use cleaning then parse
        cleaned = clean_json_text(json_str)
        try:
            return json.loads(cleaned)
        except:
            return {}


def flatten_json(nested, prefix=''):
    """Flatten nested JSON into a flat dict"""
    flat = {}
    if isinstance(nested, dict):
        for k, v in nested.items():
            key = f"{prefix}{k}" if prefix else k
            if isinstance(v, (dict, list)):
                flat.update(flatten_json(v, f"{key}."))
            else:
                flat[key] = str(v) if v is not None else ''
    elif isinstance(nested, list):
        for i, item in enumerate(nested):
            key = f"{prefix}{i}"
            if isinstance(item, (dict, list)):
                flat.update(flatten_json(item, f"{key}."))
            else:
                flat[key] = str(item) if item is not None else ''
    return flat


def iter_jsonl_records(fp, cleaner=None):
    """
    Read JSONL records robustly, buffering multiline entries.
    cleaner: function to pre-clean raw text.
    """
    decoder = json.JSONDecoder()
    buffer = ''
    for line in fp:
        buffer += line
        text = cleaner(buffer) if cleaner else buffer
        idx = 0
        while True:
            try:
                obj, idx = decoder.raw_decode(text)
                yield obj
                text = text[idx:].lstrip()
                buffer = text
            except json.JSONDecodeError:
                break
    # Final attempt
    text = cleaner(buffer) if cleaner else buffer
    try:
        obj, _ = decoder.raw_decode(text)
        yield obj
    except json.JSONDecodeError:
        pass


def parse_jsonl_to_csv(input_file, output_file, max_rows=None):
    """Parse JSONL to flattened CSV using robust cleaning."""
    rows, keys, errors = [], set(), 0
    with open(input_file, 'r', encoding='utf-8') as f:
        total = sum(1 for _ in f)
        f.seek(0)
        for i, data in enumerate(tqdm(iter_jsonl_records(f, cleaner=clean_json_text),
                                      total=total, desc='Reading')):
            if max_rows and i >= max_rows:
                break
            try:
                # apply summary parsing if present
                if 'summary' in data and isinstance(data['summary'], str):
                    data['summary'] = process_json_string(data['summary'])
                flat = flatten_json(data)
                keys.update(flat.keys())
                rows.append(flat)
            except Exception:
                errors += 1
    # write CSV
    fieldnames = sorted(keys)
    with open(output_file, 'w', newline='', encoding='utf-8') as csvf:
        writer = csv.DictWriter(csvf, fieldnames=fieldnames)
        writer.writeheader()
        for row in tqdm(rows, desc='Writing'):
            for k in fieldnames:
                row.setdefault(k, '')
            writer.writerow(row)
    print(f"Done: {len(rows)} rows, {errors} errors.")


def parse_jsonl_to_csv_preserved(input_file, output_file, max_rows=None):
    """Parse JSONL to CSV preserving top-level JSON as strings."""
    rows, fields, errors = [], set(), 0
    with open(input_file, 'r', encoding='utf-8') as f:
        total = sum(1 for _ in f)
        f.seek(0)
        for i, data in enumerate(tqdm(iter_jsonl_records(f, cleaner=clean_json_text),
                                      total=total, desc='Reading')):
            if max_rows and i >= max_rows:
                break
            try:
                row = {}
                for k, v in data.items():
                    if isinstance(v, (dict, list)):
                        row[k] = json.dumps(v, ensure_ascii=False)
                    else:
                        row[k] = str(v) if v is not None else ''
                fields.update(row.keys())
                rows.append(row)
            except Exception:
                errors += 1
    fieldnames = sorted(fields)
    with open(output_file, 'w', newline='', encoding='utf-8') as csvf:
        writer = csv.DictWriter(csvf, fieldnames=fieldnames)
        writer.writeheader()
        for row in tqdm(rows, desc='Writing'):
            for k in fieldnames:
                row.setdefault(k, '')
            writer.writerow(row)
    print(f"Done: {len(rows)} rows, {errors} errors.")


def parse_json_series(series):
    """Parse a pandas Series of JSON strings, capturing failures."""
    out = []
    for item in series:
        if isinstance(item, str):
            cleaned = clean_json_text(item)
            try:
                obj = json.loads(cleaned)
                out.append(obj if isinstance(obj, dict) else {'value': obj})
            except Exception:
                out.append({'_raw_summary': item})
        else:
            out.append({'_raw_summary': str(item)})
    return out

In [4]:
# Get current working directory
cwd = os.getcwd()

In [7]:
# Parse the JSONL file and convert to CSV
if __name__ == "__main__":
    input_file = os.path.join(cwd,'data', 'raw', 'augmented_notes_30K.jsonl')
    output_file = os.path.join(cwd, 'data', 'raw', 'medical_data.csv')

    # Preserves original structure (keeps original JSON for complex fields)
    parse_jsonl_to_csv_preserved(input_file, output_file)


Writing: 100%|██████████| 30000/30000 [00:06<00:00, 4476.55it/s]

Done: 30000 rows, 0 errors.


In [8]:
df = pd.read_csv(output_file)

In [9]:
df

,conversation,full_note,idx,note,summary
0,"Doctor: Good morning, what brings you to the Outpatient department today?\nPatient: Good morning doctor, I have some discomfort in my neck and lower back, and I'm not able to maintain an erect posture.\nDoctor: Hmm, okay. Can you tell me more about the discomfort?\nPatient: Yes, I tend to fall on either side when I stand up from a sitting position, and my head is always turned to the right and upwards.\nDoctor: I see. Are you experiencing any pain in your neck?\nPatient: Yes, I have pain and...","A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal pos...",155216,"A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal pos...","{\n""visit motivation"": ""Discomfort in the neck and lower back, restriction of body movements, inability to maintain an erect posture, and requiring assistance in standing and walking."",\n""admission"": [\n{\n""reason"": ""None"",\n""date"": ""None"",\n""duration"": ""None"",\n""care center details"": ""None""\n}\n],\n""patient information"": {\n""age"": ""Sixteen years old"",\n""sex"": ""Female"",\n""ethnicity"": ""None"",\n""weight"": ""None"",\n""height"": ""None"",\n""family medical history"": ""None"",\n""recent travels"": ""None"",\n..."
1,"Doctor: Hi, how are you feeling today?\nPatient: Hi doctor, I have this dump pain on my right back and a swelling right there for several weeks.\nDoctor: Hmm, I see. Can you describe the pain a little more?\nPatient: It's like a dull ache and it hurts more when I move.\nDoctor: Okay. I understand. And you said you're in good state and very active?\nPatient: Yes, I am.\nDoctor: That's good to hear. Have you had any health problems in the past?\nPatient: Well, I had a thoracic trauma at work o...","This is the case of a 56-year-old man that was complaining of a dump pain on the right back and a swelling right in this place for several weeks. The patient was in good state and very active. There was not any health problem in the past except a thoracic trauma at work one year ago. In that time the patient was diagnosed with a simple fracture of the 9th right rib without any other consequences.\nOn the X-ray was seen a shadow in the lower part of the right hemithorax. After that, it was de...",77465,"This is the case of a 56-year-old man that was complaining of a dump pain on the right back and a swelling right in this place for several weeks. The patient was in good state and very active. There was not any health problem in the past except a thoracic trauma at work one year ago. In that time the patient was diagnosed with a simple fracture of the 9th right rib without any other consequences.\nOn the X-ray was seen a shadow in the lower part of the right hemithorax. After that, it was de...","{\n""visit motivation"": ""Complaints of a dull pain on the right back and a swelling in the same area"",\n""admission"": [\n{\n""reason"": ""None"",\n""date"": ""None"",\n""duration"": ""None"",\n""care center details"": ""None""\n}\n],\n""patient information"": {\n""age"": ""56"",\n""sex"": ""Male"",\n""ethnicity"": ""None"",\n""weight"": ""None"",\n""height"": ""None"",

In [10]:
if 'summary' in df.columns:
     idx = df['idx'] if 'idx' in df.columns else df.index
     parsed = parse_json_series(df['summary'])
     df_sum = pd.DataFrame(parsed).fillna('')
     df_result = pd.concat([pd.Series(idx).reset_index(drop=True), df_sum.reset_index(drop=True)], axis=1)
     df_result.to_csv('parsed_summary_output.csv', index=False)

In [12]:
parsed_summary_output = os.path.join(cwd, 'data', 'clean', 'parsed_summary_output.csv')
df2 = pd.read_csv(parsed_summary_output)

In [13]:
# 1) Filter only the rows that failed parsing
fail_df = df2[df2['_raw_summary'].notna() & (df2['_raw_summary'] != '')].copy()

# 2) Compute diagnostic columns
fail_df['snippet']      = fail_df['_raw_summary'].str.slice(0, 80).str.replace('\n', '⏎')
fail_df['char_count']   = fail_df['_raw_summary'].str.len()
fail_df['quote_count']  = fail_df['_raw_summary'].str.count('"')
fail_df['braces_diff']  = (
    fail_df['_raw_summary'].str.count(r'{') - fail_df['_raw_summary'].str.count(r'}')
)
fail_df['newline_count']= fail_df['_raw_summary'].str.count('\n')

# 3) Capture the json.loads error for each
def capture_error(msg):
    try:
        json.loads(msg)
        return ''
    except Exception as e:
        return str(e)

fail_df['error_message'] = fail_df['_raw_summary'].apply(capture_error)

# 4) Compute frequency of each error and sort
fail_df['error_frequency'] = fail_df.groupby('error_message')['error_message']\
                                   .transform('count')
report = fail_df.sort_values(['error_frequency', 'char_count'],
                             ascending=[False, False])

# 5) Show the top 50 in console
print(report[['idx', 'snippet', 'char_count', 'newline_count',
              'quote_count', 'braces_diff', 'error_message',
              'error_frequency']].head(300).to_string(index=False))

# 6) Also write out to CSV for deeper inspection
report.to_csv('parsing_diagnostics.csv', index=False)
print("\nDetailed diagnostics written to parsing_diagnostics.csv")

   idx                                                                          snippet  char_count  newline_count  quote_count  braces_diff                                                                  error_message  error_frequency
  2706 {⏎"visit motivation": "Bleeding per rectum and mild abdominal pain",⏎"admission"        4946            158          468            0                         Expecting ',' delimiter: line 50 column 39 (char 1409)                3
 16046 {⏎"visit motivation": "Evaluation of severe dry eye in the left eye",⏎"admission        4511            161          468            0                         Expecting ',' delimiter: line 50 column 39 (char 1409)                3
  6566 {⏎"visit motivation": "Right elbow mass",⏎"admission": [⏎{⏎"reason": "Right elbo        2840             97          276            0                         Expecting ',' delimiter: line 50 column 39 (char 1409)                3
 89035 {⏎"visit motivation": "Recurrent disseminated

In [14]:
df3 = df2[df2['_raw_summary'].isna()].copy()


In [15]:
df3.to_csv('final_parsed_dataset.csv')